In [ ]:
import kagglehub, os
dataset_path = kagglehub.dataset_download("kondwani/eye-disease-dataset")
print("Path to dataset:", dataset_path)

100%|██████████| 4.96M/4.96M [00:00<00:00, 5.40MB/s]

Extracting files...


Path to dataset: /root/.cache/kagglehub/datasets/kondwani/eye-disease-dataset/versions/1


In [ ]:
import numpy as np

class Conv2D:
    def __init__(self, in_ch, out_ch, k, stride=1, pad=0):
        self.stride, self.pad = stride, pad
        self.weight = np.random.randn(out_ch, in_ch, k, k) * 0.1
        self.bias   = np.zeros(out_ch)
    def forward(self, x):
        C_out, C_in, k, _ = self.weight.shape
        H, W = x.shape[1], x.shape[2]
        Ph, Pw = self.pad, self.pad
        x_p = np.pad(x, ((0,0),(Ph,Ph),(Pw,Pw)), 'constant')
        H_out = (H + 2*Ph - k)//self.stride + 1
        W_out = (W + 2*Pw - k)//self.stride + 1
        y = np.zeros((C_out, H_out, W_out))
        for oc in range(C_out):
            for ic in range(C_in):
                for i in range(H_out):
                    for j in range(W_out):
                        y[oc,i,j] += np.sum(
                            x_p[ic, i*self.stride:i*self.stride+k, j*self.stride:j*self.stride+k] *
                            self.weight[oc,ic]
                        )
            y[oc] += self.bias[oc]
        return y

class DepthwiseConv2D:
    def __init__(self, in_ch, k, stride=1, pad=0):
        self.stride, self.pad = stride, pad
        self.weight = np.random.randn(in_ch, k, k) * 0.1
        self.bias   = np.zeros(in_ch)
    def forward(self, x):
        C, H, W = x.shape
        Ph, Pw = self.pad, self.pad
        x_p = np.pad(x, ((0,0),(Ph,Ph),(Pw,Pw)), 'constant')
        k = self.weight.shape[1]
        H_out = (H + 2*Ph - k)//self.stride + 1
        W_out = (W + 2*Pw - k)//self.stride + 1
        y = np.zeros((C, H_out, W_out))
        for c in range(C):
            for i in range(H_out):
                for j in range(W_out):
                    y[c,i,j] = (x_p[c, i*self.stride:i*self.stride+k, j*self.stride:j*self.stride+k] *
                                self.weight[c]).sum() + self.bias[c]
        return y

class PointwiseConv2D:
    def __init__(self, in_ch, out_ch):
        self.weight = np.random.randn(out_ch, in_ch, 1, 1) * 0.1
        self.bias   = np.zeros(out_ch)
    def forward(self, x):
        C_out, C_in, _, _ = self.weight.shape
        H, W = x.shape[1], x.shape[2]
        y = np.zeros((C_out, H, W))
        for oc in range(C_out):
            for ic in range(C_in):
                y[oc] += x[ic] * self.weight[oc,ic,0,0]
            y[oc] += self.bias[oc]
        return y

class ReLU:
    def forward(self, x):
        return np.maximum(0, x)

class GlobalAvgPool:
    def forward(self, x):
        return x.mean(axis=(1,2))

class Dense:
    def __init__(self, in_f, out_f):
        self.weight = np.random.randn(out_f, in_f) * 0.1
        self.bias   = np.zeros(out_f)
    def forward(self, x):
        return self.weight.dot(x) + self.bias

class MobileNetV1:
    def __init__(self, num_classes=3):
        self.layers = []
        self.layers += [Conv2D(3,32,3,stride=2,pad=1), ReLU()]
        in_ch = 32
        for out_ch in [64,128,256]:
            self.layers += [
                DepthwiseConv2D(in_ch,3,stride=1,pad=1), ReLU(),
                PointwiseConv2D(in_ch,out_ch), ReLU()
            ]
            in_ch = out_ch
        self.layers += [GlobalAvgPool(), Dense(in_ch, num_classes)]

    def forward(self, img):
        x = img.transpose(2,0,1)
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [ ]:
import os

dataset_path = '/root/.cache/kagglehub/datasets/kondwani/eye-disease-dataset/versions/1'
DATASET_ROOT = os.path.join(dataset_path, "Eye_diseases")

SMALL_COUNT = 10

class_names = sorted(os.listdir(DATASET_ROOT))
filepaths = []
labels    = []
for idx, cls in enumerate(class_names):
    cls_dir = os.path.join(DATASET_ROOT, cls)
    for fn in os.listdir(cls_dir):
        if fn.lower().endswith(('.png','.jpg','.jpeg')):
            filepaths.append(os.path.join(cls_dir, fn))
            labels.append(idx)

if len(filepaths) > SMALL_COUNT:
    np.random.seed(42)
    sel = np.random.choice(len(filepaths), SMALL_COUNT, replace=False)
    filepaths = [filepaths[i] for i in sel]
    labels    = [labels[i]    for i in sel]

print(f"Running on {len(filepaths)} / {len(labels)} images")
print("Classes:", class_names)

Running on 10 / 10 images
Classes: ['Bulging_Eyes', 'Cataracts', 'Crossed_Eyes', 'Glaucoma', 'Uveitis']


In [ ]:

from PIL import Image
import pandas as pd
import numpy as np

model = MobileNetV1(num_classes=len(class_names))

results = []
for path, true_idx in zip(filepaths, labels):
    img = Image.open(path).resize((224,224))
    arr = np.array(img).astype(np.float32) / 255.0
    logits = model.forward(arr)
    exps = np.exp(logits)
    probs = exps / np.sum(exps)
    pred_idx = int(np.argmax(probs))
    results.append({
        "filename": os.path.basename(path),
        "true_label": class_names[true_idx],
        "pred_label": class_names[pred_idx],
        "probability": float(probs[pred_idx])
    })

df_results = pd.DataFrame(results)
df_results.head(10)


,filename,true_label,pred_label,probability
0,images-32.jpeg,Glaucoma,Cataracts,0.200171
1,images-63.jpeg,Crossed_Eyes,Cataracts,0.200096
2,images-51.jpeg,Glaucoma,Glaucoma,0.200158
3,images-47.jpeg,Glaucoma,Cataracts,0.200198
4,image-2.jpeg,Cataracts,Glaucoma,0.200149
5,image-12.jpeg,Uveitis,Cataracts,0.200134
6,images-39.jpeg,Cataracts,Cataracts,0.200164
7,images-32.jpeg,Crossed_Eyes,Glaucoma,0.200068
8,images-25.jpeg,Uveitis,Glaucoma,0.200103
9,images-95.jpeg,Crossed_Eyes,Glaucoma,0.200074
